# 交互式geomdl示例

###  直接修改geomdl源代码实现 

In [1]:
import os
from geomdl import BSpline2 as BSpline # 重写了geomdl的三个文件：abstract2, BSpline2, VisMPL2
from geomdl import utilities
from geomdl import exchange
%matplotlib qt5 
import matplotlib.pyplot as plt
from geomdl.visualization import VisMPL2 as VisMPL
# from geomdl.visualization import VisVTK

import numpy as np



# Create a B-Spline curve instance
curve = BSpline.Curve()

# Set up the curve
curve.degree = 4
curve.ctrlpts = exchange.import_txt("ex_curve01.cpt")
# print(curve.ctrlpts)

# Auto-generate knot vector
curve.knotvector = utilities.generate_knot_vector(curve.degree, len(curve.ctrlpts))

# Set evaluation delta
curve.delta = 0.01

# Evaluate curve
curve.evaluate()

# Plot the control point polygon and the evaluated curve
vis_comp = VisMPL.VisCurve2D()
curve.vis = vis_comp
# fig = plt.figure(figsize=vis_comp.vconf.figure_size, dpi=vis_comp.vconf.figure_dpi)
fig = curve.render(reusefig = False)  # curve.render 不是vis.render


# from 这里，是交互用的

# fig = plt.gcf()
# ax = fig.gca()
# print(fig)
# print(ax)

def searchpoint(screenx,screeny,x,y,epison=20):
    '''
    像素级搜索
    screenx,screeny, 待搜索的全部点的坐标
    x，y：鼠标点的像素坐标
    '''
    datalen = len(screenx)
    ptid = None
    for k in range(datalen):
        if np.abs(screenx[k]- x) < epison and np.abs(screeny[k] - y) < epison:
            ptid = k
            break    
    return ptid

g_ptid = None
pts = np.array(curve.ctrlpts)
print(pts.shape)

def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))
    global g_ptid,pts,fig
    ax = fig.gca()
    ptsScreen = ax.transData.transform(pts)
    g_ptid = searchpoint(ptsScreen[:,0],ptsScreen[:,1],event.x,event.y) # 搜索像素坐标
    print('global ptid:',g_ptid)
cid = fig.canvas.mpl_connect('button_press_event', onclick)

def onmousemove(event):
    if event.button == None:
        return
    global g_ptid,pts,curve,fig
#     print('%s mouse move: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
#           ('double' if event.dblclick else 'single', event.button,
#            event.x, event.y, event.xdata, event.ydata))
    if None == g_ptid:
        return
    pts[g_ptid][0] = event.xdata
    pts[g_ptid][1] = event.ydata
    
#     print(pts.tolist())
    curve.ctrlpts = pts.tolist()
#     print(curve.ctlpts)
    curve.evaluate()
    ax = fig.gca()
    ax.cla()
    curve.render(reusefig = True,fig = fig)
#     ax.cla()
    fig.canvas.draw_idle()
cid = fig.canvas.mpl_connect('motion_notify_event',onmousemove)

def onrelease(event):
    print('%s release: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))
    global g_ptid
    if None != g_ptid:
        print('ptid:',g_ptid)
        g_ptid = None
    else:
        print('ptid is None')
cid = fig.canvas.mpl_connect('button_release_event',onrelease)



# Good to have something here to put a breakpoint
pass

(9, 2)
single click: button=1, x=835, y=1055, xdata=20.088660, ydata=40.110850
global ptid: 8
single release: button=1, x=712, y=1232, xdata=0.180964, ydata=0.755123
ptid: 8
single click: button=1, x=1737, y=1502, xdata=59.870266, ydata=59.836355
global ptid: 6
single release: button=1, x=1882, y=1542, xdata=113.938944, ydata=115.021323
ptid: 6
single click: button=1, x=1370, y=672, xdata=70.025953, ydata=40.476342
global ptid: 5
single release: button=1, x=2097, y=602, xdata=158.905546, ydata=33.313194
ptid: 5
single click: button=1, x=497, y=765, xdata=15.148740, ydata=47.913494
global ptid: 8
single release: button=1, x=615, y=1290, xdata=0.169355, ydata=0.811451
ptid: 8
single click: button=1, x=717, y=895, xdata=34.915301, ydata=59.593735
global ptid: 7
single release: button=1, x=1235, y=1175, xdata=81.411643, ydata=84.751176
ptid: 7
single click: button=1, x=662, y=570, xdata=29.973661, ydata=30.393134
global ptid: 2
single release: button=1, x=915, y=785, xdata=52.660282, ydata